- 練習把台北的職缺抓下來
- 完整版可以參考https://tlyu0419.github.io/2020/06/19/Crawler-104HumanResource/#more

# 載入套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
import gc
# from IPython.display import clear_output
# from tomorrow import threads
import pickle
# import grequests

In [2]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
         'Referer': 'https://www.104.com.tw/',
         'Host': 'www.104.com.tw',
         'DNT':'1',
         'Cookie':'GoogleBot'}

# 職缺清單
- 透過地區與職務分類來請求職缺資料

In [3]:
# 地區代碼
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1.head()

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005


# 觀察網頁結構
1. 台北市的地區號碼位在哪裡? 6001001001 - 6001001012
2. https://www.104.com.tw/jobs/search/?ro=2&isnew=30&area=6001001007&order=4&asc=1&page=1&mode=l&jobsource=2018indexpoc

以上設定了哪些條件?

# 練習抓一個地區一個分頁
- 並且要整理成dataframe格式
- 抓到資料後要用觀察的方式找到英文欄位對應的中文說明

In [40]:
url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=5&mode=l&jobsource=2018indexpoc'
resp = requests.get(url, headers=headers)
resp.json()['data']['list']

[{'jobType': '0',
  'jobNo': '11541019',
  'jobName': '業務行政專員',
  'jobNameSnippet': '業務行政專員',
  'jobRole': '1',
  'jobRo': '1',
  'jobAddrNo': '6001001007',
  'jobAddrNoDesc': '台北市信義區',
  'jobAddress': '忠孝東路五段508號27樓',
  'description': '1.銷售行政事務如簽約,收款,銀行對保,交屋,過戶等工作。\r\n2.客戶服務窗與客戶關係維護聯繫作業。\r\n3.銷售文書作業，如收款報表作業,合約核對製作等。\r\n4.配合主管交辦事項辦理。\r\n6.具建設公司業務銷售行政經驗者佳。\r\n7.瞭解建設公司作業流程,需具備相當熟悉業務行政流程,電腦文書作業WORD,EXCEL,POWER POINT以及高益系統懂操作。\r\n7.跨部門協調。',
  'optionEdu': '大學',
  'period': '03',
  'periodDesc': '2年',
  'applyCnt': '00037',
  'applyDesc': '30人以上應徵',
  'custNo': '96964262000',
  'custName': '吉美建設事業股份有限公司',
  'coIndustry': '1011001002',
  'coIndustryDesc': '建築工程業',
  'salaryLow': '0038000',
  'salaryHigh': '9999999',
  'salaryDesc': '月薪38,000元以上',
  's10': '50',
  'appearDate': '20201110',
  'appearDateDesc': '11/10',
  'optionZone': '0',
  'isApply': '0',
  'applyDate': '',
  'isSave': '0',
  'descSnippet': '1.銷售行政事務如簽約,收款,銀行對保,交屋,過戶等工作。\r\n2.客戶服務窗與客戶關係維護聯繫作業。\r\n3.銷售文書作業，如收款報表作業,合約核對製作等。\r\

In [41]:
pd.DataFrame(resp.json()['data']['list'])

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,isSave,descSnippet,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat
0,0,11541019,業務行政專員,業務行政專員,1,1,6001001007,台北市信義區,忠孝東路五段508號27樓,"1.銷售行政事務如簽約,收款,銀行對保,交屋,過戶等工作。\r\n2.客戶服務窗與客戶關係維...",...,0,"1.銷售行政事務如簽約,收款,銀行對保,交屋,過戶等工作。\r\n2.客戶服務窗與客戶關係維...",[員工50人],距捷運永春站260公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,業務行政專員,吉美建設事業股份有限公司,121.578548,25.040746
1,0,9672899,【雷亞概念】遊戲週邊商品企劃,【雷亞概念】遊戲週邊商品企劃,1,1,6001001007,台北市信義區,東興路45號1樓,"&quot;LIFE WITHIN GAME, GAME WITHIN LIFE&quot;...",...,0,"&quot;LIFE WITHIN GAME, GAME WITHIN LIFE&quot;...",[員工200人],距捷運南京三民站470公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,【雷亞概念】遊戲週邊商品企劃,雷亞遊戲股份有限公司,121.5662139,25.047832
2,2,10739175,"Specialist, CIEM 資深客戶關係管理專員","Specialist, CIEM 資深客戶關係管理專員",1,1,6001001007,台北市信義區,,In-store event plan &amp; execution: \n - Coop...,...,0,In-store event plan &amp; execution: \n - Coop...,"[外商公司, 員工270人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,"Specialist, CIEM 資深客戶關係管理專員",香港商雷夫羅倫有限公司台灣分公司( Ralph Lauren (Hong Kong) Ret...,121.5769572,25.0287024
3,2,11733180,風控管理師,風控管理師,1,1,6001001007,台北市信義區,忠孝東路五段68號35樓,1. 風控系統數據日常監控與風險事件檢查\r\n2. 對業務資料進行深入挖掘、開發風險和市場...,...,0,1. 風控系統數據日常監控與風險事件檢查\r\n2. 對業務資料進行深入挖掘、開發風險和市場...,[],距捷運市政府站110公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,風控管理師,大牛科技有限公司,121.567154,25.040512
4,2,8113714,軟體程式設計師 Software Engineer,軟體程式設計師 Software Engineer,1,1,6001001007,台北市信義區,基隆路二段51號3樓之6,35年來神華在零售業、流通業與製造業一直具有著領導地位，神華從當年台灣第一家廠商成功將POS...,...,0,35年來神華在零售業、流通業與製造業一直具有著領導地位，神華從當年台灣第一家廠商成功將POS...,[員工50人],距捷運台北101/世貿站490公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,軟體程式設計師 Software Engineer,神華商業系統股份有限公司,121.558469,25.0309321
5,0,11928499,全端開發工程師(研發專案),全端開發工程師(研發專案),1,1,6001001007,台北市信義區,,1. 具備Javascript/JQuery/CSS/HTML前端實務開發經驗\n2. 熟悉...,...,0,1. 具備Javascript/JQuery/CSS/HTML前端實務開發經驗\n2. 熟悉...,[員工80人],距捷運永春站410公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,全端開發工程師(研發專案),易勝資訊股份有限公司,121.5720055,25.0409201
6,0,11623475,業務行政助理,業務行政助理,1,1,6001001007,台北市信義區,基隆路一段163號11樓之1,1. 協助業務人員處理銷售業務相關之公司內部行政作業及其他交辦事項 \r\n2. 協助業務人...,...,0,1. 協助業務人員處理銷售業務相關之公司內部行政作業及其他交辦事項 \r\n2. 協助業務人...,[],距捷運市政府站130公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,業務行政助理,華威數位財金科技股份有限公司,121.565287,25.041788
7,0,4450169,服務工程師,服務工程師,1,1,6001001007,台北市信義區,基隆路一段155號6樓,1.精密儀器安裝\r\n2.精密儀器維修服務\r\n3.提供客戶技術服務\r\n4.處理系統...,...,0,1.精密儀器安裝\r\n2.精密儀器維修服務\r\n3.提供客戶技術服務\r\n4.處理系統...,[員工30人],距捷運市政府站150公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,服務工程師,博精儀器股份有限公司,121.565396,25.042178
8,0,11871347,(中班) 線上文字客服人員,(中班) 線上文字客服人員,1,1,6001001007,台北市信義區,松勤街86號,"公司福利：\n●年節獎金\n- 生日禮金 $2,000\n- 三節禮金或禮品：端午、中秋、農...",...,0,"公司福利：\n●年節獎金\n- 生日禮金 $2,000\n- 三節禮金或禮品：端午、中秋、農...",[員工30人],距捷運台北101/世貿站250公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,(中班) 線上文字客服人員,涵澤有限公司,121.5648143,25.0317902
9,0,8213709,美術/畫畫老師（全職）,美術/畫畫老師（全職）,1,1,6001001007,台北市信義區,松勤街41號1樓,1. 對藝術教學有極大熱忱，能善用壓克力，黏土，多媒材，素描教學。\n2.企業活動／大人小孩...,...,0,1. 對藝術教學有極大熱忱，能善用壓克力，黏土，多媒材，素描教學。\n2.企業活動／大人小孩...,[],距捷運台北101/世貿站100公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,美術/畫畫老師（全職）,畫飲藝術文創有限公司,121.5628169,25.0321669


# 練習抓一個地區，100個分頁

## 怎麼串接資料表?
- pd.concat

## str.format

In [15]:
for page in range(10):
    url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(page+1)
    print(url)

https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=1&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=2&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=3&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=4&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=5&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=6&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=7&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page=8&mode=l&jobsource=2018indexpoc
https://www.104.com.tw/j

## 抓100個分頁
- 先不給timesleep，等到發生錯誤的時候再跟他們說要加入時間機制

In [19]:
df = []
for page in range(100):
    time.sleep(0.5)
    url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area=6001001007&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(page+1)
    resp = requests.get(url, headers=headers)
    ndf = pd.DataFrame(resp.json()['data']['list'])
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,isSave,descSnippet,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat
0,1,11928048,軟體開發技術主管(內部App/ Web專案開發) Software Development ...,軟體開發技術主管(內部App/ Web專案開發) Software Development ...,4,1,6001001007,台北市信義區,市民大道六段288號8樓之8,《工作說明》：\n\n1. 具專案領導或系統開發整合經驗，APP/WEB專案開發尤佳。\n\...,...,0,《工作說明》：\n\n1. 具專案領導或系統開發整合經驗，APP/WEB專案開發尤佳。\n\...,[員工80人],距捷運松山站170公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,軟體開發技術主管(內部App/ Web專案開發) Software Development ...,智鈦星平台服務有限公司,121.5790844,25.0492674
1,1,11815941,"Operations Manager, Technical Premium Support","Operations Manager, Technical Premium Support",4,1,6001001007,台北市信義區,松仁路32號,[About Amazon Web Services]\nAmazon&#039;s tec...,...,0,[About Amazon Web Services]\nAmazon&#039;s tec...,[],距捷運市政府站330公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,"Operations Manager, Technical Premium Support",Amazon Web Services Taiwan Limited_台灣亞馬遜網路服務有限公司,121.5682107,25.0387163
2,1,10149712,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,1,1,6001001007,台北市信義區,(依照公司規定分派工作地點),【信義房屋】30天工作鑑賞期\n把你的未來，放在公司最前面\n讓你成為最專業不動產經紀人員\...,...,0,【信義房屋】30天工作鑑賞期\n把你的未來，放在公司最前面\n讓你成為最專業不動產經紀人員\...,"[上市上櫃, 員工5000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,信義房屋股份有限公司,121.5769572,25.0287024
3,1,11808332,Cloud Support Engineer – Security,Cloud Support Engineer – Security,4,1,6001001007,台北市信義區,松仁路32號,[About Amazon Web Services]\nAmazon Web Servic...,...,0,[About Amazon Web Services]\nAmazon Web Servic...,[],距捷運市政府站330公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,Cloud Support Engineer – Security,Amazon Web Services Taiwan Limited_台灣亞馬遜網路服務有限公司,121.5682107,25.0387163
4,1,10564103,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,1,1,6001001007,台北市信義區,(依照公司規定分派工作地點),【信義房屋】30天工作鑑賞期\n把你的未來，放在公司最前面\n讓你成為最專業不動產經紀人員\...,...,0,【信義房屋】30天工作鑑賞期\n把你的未來，放在公司最前面\n讓你成為最專業不動產經紀人員\...,"[上市上櫃, 員工5000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,【信義房屋】雙北地區_不動產經紀人員，30天工作體驗，不適應離職再5萬，立即預約！,信義房屋股份有限公司,121.5769572,25.0287024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,0,11364697,網路行銷企劃,網路行銷企劃,1,1,6001001007,台北市信義區,和平東路三段441號1樓,徵求社群行銷企劃：\r\n歡迎對於女性產業有熱忱的你，成為我們的夥伴！\r\n\r\n▲專案...,...,0,徵求社群行銷企劃：\r\n歡迎對於女性產業有熱忱的你，成為我們的夥伴！\r\n\r\n▲專案...,[員工30人],距捷運麟光站60公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,網路行銷企劃,蘿琳亞股份有限公司,121.5585321,25.0189897
3004,2,7624034,(Crate&amp;Barrel 微風信義店) 家具銷售專員,(Crate&amp;Barrel 微風信義店) 家具銷售專員,1,1,6001001007,台北市信義區,微風信義店,Crate&amp;Barrel，來自美國芝加哥的簡約當代風格家俱家飾領導品牌，不斷為提升顧...,...,0,Crate&amp;Barrel，來自美國芝加哥的簡約當代風格家俱家飾領導品牌，不斷為提升顧...,[員工3700人],距捷運市政府站90公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,(Crate&Barrel 微風信義店) 家具銷售專員,特力集團＿HOLA＿HOLA CASA＿HOLA Petite＿Crate&Barrel＿特...,121.5669858,25.0405694
3005,0,11290234,樺達奶茶_門市人員(信義大遠百店),樺達奶茶_門市人員(信義大遠百店),1,1,6001001007,台北市信義區,大遠百,1.百貨及街邊店飲料販售\r\n2.飲料調製\r\n3.結帳與收銀。 \r\n4.維護工作環...,...,0,1.百貨及街邊店飲料販售\r\n2.飲料調製\r\n3.結帳與收銀。 \r\n4.維護工作環...,[],距捷運市政府站180公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,樺達奶茶_門市人員(信義大遠百店),樺達奶茶,121.5653771,25.0397146
3006,2,10642098,知名品牌營養品面銷人員(台北),知名品牌營養品面銷人員(台北),1,1,6001001007,台北市信義區,台北市各大藥局(依排班而定),1.營養衛教諮詢及產品介紹\r\n2.顧客維繫服務及客訴案件處理\r\n3.紀錄並維護顧客資...,...,0,1.營養衛教諮詢及產品介紹\r\n2.顧客維繫服務及客訴案件處理\r\n3.紀錄並維護顧客資...,[員工30人],距捷運永春站440公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,知名品牌營養品面銷人員(台北),雉羽整合行銷股份有限公司,121.5789032,25.037943


# 練習把台北市各區的資料都抓下來
- 把上面的程式碼複製下來

## 正常版

In [24]:
areas = ['6001001001', '6001001002', '6001001003', '6001001004', '6001001005', '6001001006', 
         '6001001007', '6001001008', '6001001009', '6001001010', '6001001011', '6001001012']

df = []
for area in areas:    
    for page in range(100):
#         time.sleep(1)
        url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area={}&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(area, page+1)
        resp = requests.get(url, headers=headers)
        ndf = pd.DataFrame(resp.json()['data']['list'])
        print(area, page, ndf.shape)
        df.append(ndf)
df = pd.concat(df, ignore_index=True)
df

6001001001 0 (40, 38)
6001001001 1 (30, 38)
6001001001 2 (30, 38)
6001001001 3 (30, 38)
6001001001 4 (30, 38)
6001001001 5 (30, 38)
6001001001 6 (30, 38)
6001001001 7 (30, 38)
6001001001 8 (30, 38)
6001001001 9 (30, 38)
6001001001 10 (30, 38)
6001001001 11 (30, 38)
6001001001 12 (30, 38)


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))

## 加速版

In [3]:
areas = ['6001001001', '6001001002', '6001001003', '6001001004', '6001001005', '6001001006', 
         '6001001007', '6001001008', '6001001009', '6001001010', '6001001011', '6001001012']

df = []
for area in areas:    
    page = 1
    while True:
        try:
            url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&area={}&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(area, page+1)
            resp = requests.get(url, headers=headers)
            ndf = pd.DataFrame(resp.json()['data']['list'])
            print(area, page, ndf.shape)
            df.append(ndf)
            page += 1
            if page == 100:
                break
        except:
            print('err')
df = pd.concat(df, ignore_index=True)
df

6001001001 1 (30, 38)
6001001001 2 (30, 38)
6001001001 3 (30, 38)
6001001001 4 (30, 38)
6001001001 5 (30, 38)
6001001001 6 (30, 38)
6001001001 7 (30, 38)
6001001001 8 (30, 38)
6001001001 9 (30, 38)
6001001001 10 (30, 38)
6001001001 11 (30, 38)
6001001001 12 (30, 38)
6001001001 13 (30, 38)
6001001001 14 (30, 38)
6001001001 15 (30, 38)
6001001001 16 (30, 38)
6001001001 17 (30, 38)
6001001001 18 (30, 38)
6001001001 19 (30, 38)
6001001001 20 (30, 38)
6001001001 21 (30, 38)
6001001001 22 (30, 38)
6001001001 23 (30, 38)
6001001001 24 (30, 38)
6001001001 25 (30, 38)
6001001001 26 (30, 38)
6001001001 27 (30, 38)
6001001001 28 (30, 38)
6001001001 29 (30, 38)
6001001001 30 (30, 38)
6001001001 31 (30, 38)
6001001001 32 (30, 38)
6001001001 33 (30, 38)
6001001001 34 (30, 38)
6001001001 35 (30, 38)
6001001001 36 (30, 38)
6001001001 37 (30, 38)
6001001001 38 (30, 38)
6001001001 39 (30, 38)
6001001001 40 (30, 38)
6001001001 41 (30, 38)
6001001001 42 (30, 38)
6001001001 43 (30, 38)
6001001001 44 (30, 3

6001001004 65 (30, 38)
6001001004 66 (30, 38)
6001001004 67 (30, 38)
6001001004 68 (30, 38)
6001001004 69 (30, 38)
6001001004 70 (30, 38)
6001001004 71 (30, 38)
6001001004 72 (30, 38)
6001001004 73 (30, 38)
6001001004 74 (30, 38)
6001001004 75 (30, 38)
6001001004 76 (30, 38)
6001001004 77 (30, 38)
6001001004 78 (30, 38)
6001001004 79 (30, 38)
6001001004 80 (30, 38)
6001001004 81 (30, 38)
6001001004 82 (30, 38)
6001001004 83 (30, 38)
6001001004 84 (30, 38)
6001001004 85 (30, 38)
6001001004 86 (30, 38)
6001001004 87 (30, 38)
6001001004 88 (30, 38)
6001001004 89 (30, 38)
6001001004 90 (30, 38)
6001001004 91 (30, 38)
6001001004 92 (30, 38)
6001001004 93 (30, 38)
6001001004 94 (30, 38)
6001001004 95 (30, 38)
6001001004 96 (30, 38)
6001001004 97 (30, 38)
6001001004 98 (30, 38)
6001001004 99 (30, 38)
6001001005 1 (30, 38)
6001001005 2 (30, 38)
6001001005 3 (30, 38)
6001001005 4 (30, 38)
6001001005 5 (30, 38)
6001001005 6 (30, 38)
6001001005 7 (30, 38)
6001001005 8 (30, 38)
6001001005 9 (30, 3

6001001008 33 (30, 38)
6001001008 34 (30, 38)
6001001008 35 (30, 38)
6001001008 36 (30, 38)
6001001008 37 (30, 38)
6001001008 38 (30, 38)
6001001008 39 (30, 38)
6001001008 40 (30, 38)
6001001008 41 (30, 38)
6001001008 42 (30, 38)
6001001008 43 (30, 38)
6001001008 44 (30, 38)
6001001008 45 (30, 38)
6001001008 46 (30, 38)
6001001008 47 (30, 38)
6001001008 48 (30, 38)
6001001008 49 (30, 38)
6001001008 50 (30, 38)
6001001008 51 (30, 38)
6001001008 52 (30, 38)
6001001008 53 (30, 38)
6001001008 54 (30, 38)
6001001008 55 (30, 38)
6001001008 56 (30, 38)
6001001008 57 (30, 38)
6001001008 58 (30, 38)
6001001008 59 (30, 38)
6001001008 60 (7, 38)
6001001008 61 (0, 0)
6001001008 62 (0, 0)
6001001008 63 (0, 0)
6001001008 64 (0, 0)
6001001008 65 (0, 0)
6001001008 66 (0, 0)
6001001008 67 (0, 0)
6001001008 68 (0, 0)
6001001008 69 (0, 0)
6001001008 70 (0, 0)
6001001008 71 (0, 0)
6001001008 72 (0, 0)
6001001008 73 (0, 0)
6001001008 74 (0, 0)
6001001008 75 (0, 0)
6001001008 76 (0, 0)
6001001008 77 (0, 0)


6001001012 3 (30, 38)
6001001012 4 (30, 38)
6001001012 5 (30, 38)
6001001012 6 (30, 38)
6001001012 7 (30, 38)
6001001012 8 (30, 38)
6001001012 9 (30, 38)
6001001012 10 (30, 38)
6001001012 11 (30, 38)
6001001012 12 (30, 38)
6001001012 13 (30, 38)
6001001012 14 (30, 38)
6001001012 15 (30, 38)
6001001012 16 (30, 38)
6001001012 17 (30, 38)
6001001012 18 (30, 38)
6001001012 19 (30, 38)
6001001012 20 (30, 38)
6001001012 21 (30, 38)
6001001012 22 (30, 38)
6001001012 23 (30, 38)
6001001012 24 (30, 38)
6001001012 25 (30, 38)
6001001012 26 (16, 38)
6001001012 27 (0, 0)
6001001012 28 (0, 0)
6001001012 29 (0, 0)
6001001012 30 (0, 0)
6001001012 31 (0, 0)
6001001012 32 (0, 0)
6001001012 33 (0, 0)
6001001012 34 (0, 0)
6001001012 35 (0, 0)
6001001012 36 (0, 0)
6001001012 37 (0, 0)
6001001012 38 (0, 0)
6001001012 39 (0, 0)
6001001012 40 (0, 0)
6001001012 41 (0, 0)
6001001012 42 (0, 0)
6001001012 43 (0, 0)
6001001012 44 (0, 0)
6001001012 45 (0, 0)
6001001012 46 (0, 0)
6001001012 47 (0, 0)
6001001012 48 

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,isSave,descSnippet,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat
0,2,9071033,電話行銷人員-捷運西門站(電話行銷二部),電話行銷人員-捷運西門站(電話行銷二部),1,1,6001001001,台北市中正區,寶慶路37號3F、45號1、2樓,【工作內容】\r\n1. 提供會員服務及商品活動訊息。\r\n2. 透過電話行銷創造客戶需求...,...,0,【工作內容】\r\n1. 提供會員服務及商品活動訊息。\r\n2. 透過電話行銷創造客戶需求...,[員工2200人],距捷運西門站210公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,電話行銷人員-捷運西門站(電話行銷二部),東森購物_電視購物_東森得易購股份有限公司,121.510272,25.041692
1,0,11810397,國外選品企劃採購專員 / International Merchandiser,國外選品企劃採購專員 / International Merchandiser,1,1,6001001001,台北市中正區,八德路1段23號15樓,"職缺描述\nCompany Description\n【各地精選, 在地發現 Global ...",...,0,"職缺描述\nCompany Description\n【各地精選, 在地發現 Global ...",[員工30人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,國外選品企劃採購專員 / International Merchandiser,任開數位媒體行銷股份有限公司,121.558023,25.0484126
2,0,10286144,【Dappei】媒體業務,【Dappei】媒體業務,1,1,6001001001,台北市中正區,博愛路38號4樓,Dappei 搭配 是業界首屈的網路流行穿搭媒體\r\n\r\n我們從生活搭配中找尋樂趣，穿...,...,0,Dappei 搭配 是業界首屈的網路流行穿搭媒體\r\n\r\n我們從生活搭配中找尋樂趣，穿...,[員工90人],距捷運北門站390公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,【Dappei】媒體業務,Shopping99_耐德科技股份有限公司,121.5111264,25.0458337
3,0,11811548,【總部】財務會計助理,【總部】財務會計助理,1,1,6001001001,台北市中正區,忠孝西路一段6號4樓,1.收發、記錄及彙整交易產生之原始憑證，並登錄至會計系統製作傳票。\n2.審核有關各項費用支...,...,0,1.收發、記錄及彙整交易產生之原始憑證，並登錄至會計系統製作傳票。\n2.審核有關各項費用支...,[],距捷運台北車站250公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,【總部】財務會計助理,蓋婭基因股份有限公司,121.5180933,25.0457613
4,2,8355585,店長(薪資50000起面議),店長(薪資50000起面議),1,1,6001001001,台北市中正區,忠孝西路一段38號B1凱撒飯店,1.店鋪管理內外場QSC\r\n2.品質控管管理\r\n3.內外服務溝通\r\n4.數字成本...,...,0,1.店鋪管理內外場QSC\r\n2.品質控管管理\r\n3.內外服務溝通\r\n4.數字成本...,[員工100人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,店長(薪資50000起面議),綠洲餐飲管理顧問股份有限公司,121.5654268,25.0329636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27556,0,11603180,業務助理,業務助理,1,1,6001001012,台北市文山區,景興路153巷51號,1.輸入、處理並追蹤客戶訂單，並將訂單資料存檔備份。\r\n2.詢價/訂貨並確認交貨期。\r...,...,0,1.輸入、處理並追蹤客戶訂單，並將訂單資料存檔備份。\r\n2.詢價/訂貨並確認交貨期。\r...,[],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,業務助理,力衡室內裝修有限公司,121.5463116,24.9940085
27557,0,11554375,護理師,護理師,1,1,6001001012,台北市文山區,萬慶街18號2樓,照護透析病人 \r\n具有執照或內外科、加護病房、急診室經驗。 \r\n,...,0,照護透析病人 \r\n具有執照或內外科、加護病房、急診室經驗。 \r\n,[],距捷運景美站110公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,護理師,慶如診所,121.5397718,24.9925711
27558,0,5451310,建築設計師-【無經驗可，建築系畢/需會3D繪圖能力】,建築設計師-【無經驗可，建築系畢/需會3D繪圖能力】,1,1,6001001012,台北市文山區,萬安街103號1樓,熟悉3D繪圖，具平面規劃、建築外觀設計能力 。\r\n公司管理給予每位同仁足夠的學習與發揮的...,...,0,熟悉3D繪圖，具平面規劃、建築外觀設計能力 。\r\n公司管理給予每位同仁足夠的學習與發揮的...,[員工15人],距捷運木柵站300公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,建築設計師-【無經驗可，建築系畢/需會3D繪圖能力】,鄭復綱建築師事務所,121.571898,25.000733
27559,2,8768578,【正職人員】台北市文山區,【正職人員】台北市文山區,1,1,6001001012,台北市文山區,摩斯漢堡門市(文山區),1.營運現場各大工作站流程熟悉與管理。\n2.商品知識與食品安全概念的認識與執行。\n3.溫...,...,0,1.營運現場各大工作站流程熟悉與管理。\n2.商品知識與食品安全概念的認識與執行。\n3.溫...,"[上市上櫃, 員工5800人]",距捷運景美站350公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,【正職人員】台北市文山區,摩斯漢堡_安心食品服務股份有限公司,121.5443243,24.9924835


# 保存資料

In [4]:
# 限全職:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27561 entries, 0 to 27560
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobType         27561 non-null  object
 1   jobNo           27561 non-null  object
 2   jobName         27561 non-null  object
 3   jobNameSnippet  27561 non-null  object
 4   jobRole         27561 non-null  object
 5   jobRo           27561 non-null  object
 6   jobAddrNo       27561 non-null  object
 7   jobAddrNoDesc   27561 non-null  object
 8   jobAddress      27561 non-null  object
 9   description     27561 non-null  object
 10  optionEdu       27561 non-null  object
 11  period          27561 non-null  object
 12  periodDesc      27561 non-null  object
 13  applyCnt        27561 non-null  object
 14  applyDesc       27561 non-null  object
 15  custNo          27561 non-null  object
 16  custName        27561 non-null  object
 17  coIndustry      27561 non-null  object
 18  coIndu

In [ ]:
df.to_excel('df.xlsx')